In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("spotify_millsongdata.csv")

In [3]:
df.head(5)

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \r\nA..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \r\nTouch me gen..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \r\nWhy I had...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


In [4]:
df.tail(5)

,artist,song,link,text
57645,Ziggy Marley,Good Old Days,/z/ziggy+marley/good+old+days_10198588.html,Irie days come on play \r\nLet the angels fly...
57646,Ziggy Marley,Hand To Mouth,/z/ziggy+marley/hand+to+mouth_20531167.html,Power to the workers \r\nMore power \r\nPowe...
57647,Zwan,Come With Me,/z/zwan/come+with+me_20148981.html,all you need \r\nis something i'll believe \...
57648,Zwan,Desire,/z/zwan/desire_20148986.html,northern star \r\nam i frightened \r\nwhere ...
57649,Zwan,Heartsong,/z/zwan/heartsong_20148991.html,come in \r\nmake yourself at home \r\ni'm a ...


In [5]:
df.shape

(57650, 4)

In [6]:
df.isnull().sum()

artist    0
song      0
link      0
text      0
dtype: int64

In [7]:
df =df.sample(5000).drop('link', axis=1).reset_index(drop=True)

In [8]:
df.head(10)

,artist,song,text
0,Aerosmith,Crash,"(Dum, dum..can't find anything \r\nBetter to ..."
1,Hooverphonic,Waves,These gigantic waves just take me \r\nTo an u...
2,Hillsong United,Age To Age,[Verse 1] \r\nYou gave me hope you made me wh...
3,Keith Urban,Used To The Pain,And so I wrote this song for you \r\nI think ...
4,Kenny Rogers,Have I Told You Lately That I Love You,Have I told you lately that I love you \r\nHa...
5,Van Morrison,I Forgot That Love Existed,I forgot that love existed troubled in my mind...
6,Nicki Minaj,I Get Crazy,I just came out of the mutherfuckin old school...
7,Roy Orbison,Lana,"Zuhm-ma zhum-ma-zhum-ma, \r\nZuhm-ma zhum-ma-..."
8,Oasis,Force Of Nature,Yeah I feel like a force of nature \r\nCould ...
9,Elvis Presley,After Loving You,"But now after loving you, what else is there t..."


In [9]:
df['text'][0]

"(Dum, dum..can't find anything  \r\nBetter to do...you drive me crazy)  \r\nSitting on a castle  \r\nOn the floor with Esmeralda  \r\nWaiting for the Geezer  \r\nCaptain Crunch to come around  \r\nFeeling suicidal 'cause of certain situations  \r\nHanging from the stairs too long  \r\nI think I'm coming down  \r\nIt's time to Crash  \r\n(Losin' my mind...yeah, it's time to)  \r\nDoin' tons of colon blow  \r\nStuffing it in every hole  \r\nMile high, so am I  \r\nWatching Jimmy kiss the sky  \r\nSalle diner becomes a blur  \r\nWith everybody jammin'  \r\nRepublicans begin to sin  \r\nThe batwings start a slamming  \r\nIt's time to crash  \r\n(Losin' my mind...yeah, it's time to)  \r\nCrash  \r\n(Yeah, maybe it's a little late but it's time to)  \r\nCrash  \r\nYou drive me crazy  \r\nYou drive me crazy  \r\nYou drive me crazy  \r\nCaptain Billy Beemish  \r\nPut a lamb up on a rack  \r\nThanks for coming Billy  \r\nAnd we hope you're comin' back  \r\nThe castle will be waiting  \r\nFor t

In [10]:
df.shape

(5000, 3)

In [11]:
df['text'] = df['text'].str.lower().replace(r'^\w\s', ' ').replace(r'\n', ' ', regex = True)

In [12]:
import nltk
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

def tokenization(txt):
    tokens = nltk.word_tokenize(txt)
    stemming = [stemmer.stem(w) for w in tokens]
    return " ".join(stemming)

In [13]:
df['text'] = df['text'].apply(lambda x: tokenization(x))

In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [15]:
tfidvector = TfidfVectorizer(analyzer='word',stop_words='english')
matrix = tfidvector.fit_transform(df['text'])
similarity = cosine_similarity(matrix)

In [16]:
similarity[0]

array([1.        , 0.00140084, 0.01485003, ..., 0.08714087, 0.01807451,
       0.0618342 ])

In [17]:
df[df['song'] == 'Crying Over You']

,artist,song,text


In [30]:
def recommendation(song_df):
    idx = df[df['song'] == song_df].index[0]
    distances = sorted(list(enumerate(similarity[idx])),reverse=True,key=lambda x:x[1])
    
    songs = []
    for m_id in distances[1:21]:
        songs.append(df.iloc[m_id[0]].song)
        
    return songs

In [26]:
recommendation('Crying Over You')

IndexError: index 0 is out of bounds for axis 0 with size 0

In [27]:
import pickle
pickle.dump(similarity,open('similarity.pkl','wb'))
pickle.dump(df,open('df.pkl','wb'))